<a href="https://colab.research.google.com/github/cihankaradogan/Stylegan2-Ada-Training/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pull the latest version of stylegan2-ada from github, Use the right version of TF

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada
!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

TensorFlow 1.x selected.
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 71, done.
remote: Total 71 (delta 0), reused 0 (delta 0), pack-reused 71
Unpacking objects: 100% (71/71), done.
/content/stylegan2-ada
gcc: error: test_nvcc.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.
Tensorflow version: 1.15.2
GPU 0: Tesla K80 (UUID: GPU-5cd82b9f-5935-4381-4245-6898f03284fa)
GPU Identified at: /device:GPU:0


# Prepare the data



The below cell creates a TF records file which stylegan2 ada needs to train successfully!

In [ ]:
%cd /content/stylegan2-ada/
!python dataset_tool.py create_from_images /content/datasets/custom/ /content/drive/MyDrive/samples/
                                            #out dir                    data's dir

/content/stylegan2-ada
Loading images from "/content/drive/MyDrive/samples/"
Creating dataset "/content/datasets/custom/"
dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 101 images.


In [ ]:
%cd /content/stylegan2-ada

/content/stylegan2-ada


# Training

In [ ]:
!python train.py --outdir=/training-runs --gpus=1 --data=/content/datasets/custom \
      --cfg=paper1024

tcmalloc: large alloc 4294967296 bytes == 0x55b8f031e000 @  0x7f03a7170001 0x7f03a43b354f 0x7f03a4403b58 0x7f03a4407b17 0x7f03a44a6203 0x55b8e8e3c0a4 0x55b8e8e3bda0 0x55b8e8eb0868 0x55b8e8eab235 0x55b8e8e3dfec 0x55b8e8e7ebc9 0x55b8e8e7bac4 0x55b8e8e3c8a9 0x55b8e8eb0b0a 0x55b8e8eaac35 0x55b8e8d7ce2c 0x55b8e8ead318 0x55b8e8eaac35 0x55b8e8d7ce2c 0x55b8e8ead318 0x55b8e8eab235 0x55b8e8d7ce2c 0x55b8e8ead318 0x55b8e8e3d65a 0x55b8e8eabb0e 0x55b8e8eaac35 0x55b8e8eaa933 0x55b8e8f74402 0x55b8e8f7477d 0x55b8e8f74626 0x55b8e8f4c313
tcmalloc: large alloc 4294967296 bytes == 0x55b9f031e000 @  0x7f03a716e1e7 0x7f03a43b346e 0x7f03a4403c7b 0x7f03a440435f 0x7f03a44a6103 0x55b8e8e3c0a4 0x55b8e8e3bda0 0x55b8e8eb0868 0x55b8e8eaac35 0x55b8e8e3d73a 0x55b8e8eac93b 0x55b8e8eaac35 0x55b8e8e3d73a 0x55b8e8eac93b 0x55b8e8eaac35 0x55b8e8e3d73a 0x55b8e8eac93b 0x55b8e8e3d65a 0x55b8e8eabb0e 0x55b8e8eaac35 0x55b8e8e3d73a 0x55b8e8eaff40 0x55b8e8eaac35 0x55b8e8e3d73a 0x55b8e8eac93b 0x55b8e8eab235 0x55b8e8e3dfec 0x55b8e8e7

  --outdir DIR          Where to save the results (required)

  --gpus INT            Number of GPUs to use (default: 1 gpu)

  --data PATH           Training dataset path (required)

  --kimg INT            Override training duration

**configs (--cfg):**

  auto           Automatically select reasonable defaults based on resolution
                 and GPU count. Good starting point for new datasets.

  stylegan2      Reproduce results for StyleGAN2 config F at 1024x1024.

  paper256       Reproduce results for FFHQ and LSUN Cat at 256x256.

  paper512       Reproduce results for BreCaHAD and AFHQ at 512x512.

  paper1024      Reproduce results for MetFaces at 1024x1024.

  cifar          Reproduce results for CIFAR-10 (tuned configuration).

  cifarbaseline  Reproduce results for CIFAR-10 (baseline configuration).


**transfer learning source networks (--resume):**

  ffhq256        FFHQ trained at 256x256 resolution.

  ffhq512        FFHQ trained at 512x512 resolution.

  ffhq1024       FFHQ trained at 1024x1024 resolution.

  celebahq256    CelebA-HQ trained at 256x256 resolution.

  lsundog256     LSUN Dog trained at 256x256 resolution.

# Convert it to a bigger model

In [ ]:
%cd ..
!git clone https://github.com/aydao/stylegan2-surgery.git
%cd stylegan2-surgery

The following will create a new, untrained network with your specified resolution. We need this to copy weights into.

Change the **width** and **height** values!

In [ ]:
!python create_initial_network_pkl.py --width 4096 --height 4096

### Copying Weights
Now that we have an empty network we need to copy weights into it.

What you need to do here if you want to use this network is change the height and width in the final argument of the following command. They're in bold below...

network-initial-config-f-4096x4096-0.pkl

In [ ]:
!python copy_weights.py /training-runs/stylegan2-ffhq-config-f.pkl /content/stylegan2-surgery/network-initial-config-f-4096x4096-0.pkl --output_pkl /content/surgery_output.pkl
                            #the previous model we trained         #model with 4096 resolution that we created in the previous code              #the model we will get as output

# Generating

In [ ]:
!python generate.py --outdir=out --trunc=0.7 --seeds=600-605 \
    --network=/content/surgery_output.pkl

  --outdir DIR          Where to save the results (required)

  --seeds INT           Random number range (600-605 generates 6 images)  

  --network PATH        Path of the model